In [ ]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import shutil
import random
print(device)

In [ ]:
import os
classes_dir = os.listdir("data/object_images")
root_dir="data/folder/"
classes_dir

In [ ]:
for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls,exist_ok=True)
    os.makedirs(root_dir +'test/' + cls,exist_ok=True)

In [ ]:
def call2(filen):
    src = 'data/object_images/'+filen
    allFileNames = os.listdir(src)
    # Random shuffle the images so that we can select 5 random images.
    np.random.shuffle(allFileNames)
    test_ratio=5.0/len(allFileNames)
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* test_ratio)])
    

    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print("*****************************")
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Testing: ', len(test_FileNames))
    print("*****************************")

    for name in train_FileNames:
        shutil.copy(name, root_dir +'train/'+filen+'/' )

    for name in test_FileNames:
        shutil.copy(name, root_dir +'test/'+filen+'/')
    print("Copying Done!")

In [ ]:
for x in classes_dir:
    call2(x)

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and validation datasets
train_dataset = ImageFolder('data/folder/train', transform=transform)
val_dataset = ImageFolder('data/folder/test', transform=transform)


In [ ]:
# Define the loss function and optimizer
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False
# Modify the last layer of the model
num_classes = len(classes_dir) # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)


In [ ]:
from torch.utils.data import DataLoader

# Create data loaders for the train and validation datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [41]:
from sklearn.metrics import confusion_matrix
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            print(outputs.shape)
            _, preds = torch.max(outputs, 1)
            print(preds.shape)
            assert(False)
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

        # Set the model to evaluation mode
        model.eval()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Initialize lists to store true and predicted labels
        all_labels = []
        all_preds = []

        # Iterate over the batches of the validation loader
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move the inputs and labels to the device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Store the true and predicted labels
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_dataset)
        val_acc = running_corrects.double() / len(val_dataset)

        

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}, val loss: {:.4f}, val acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))
        # print('Confusion Matrix:\n', conf_matrix)
    return all_labels, all_preds   
all_labels, all_preds=train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

torch.Size([32, 22])
torch.Size([32])


AssertionError: 

In [ ]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Fine-tune the last layer for a few epochs
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
all_labels, all_preds=train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
conf_matrix

In [ ]:

# Unfreeze all the layers and fine-tune the entire network for a few more epochs
# for param in model.parameters():
#     param.requires_grad = True
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

In [ ]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [49]:
def calculate_centroids(class_features):
    centroids = {}
    for class_idx, features in class_features.items():
        if features:  # Check if the list of features is non-empty
            centroids[class_idx] = torch.mean(torch.stack(features), dim=0).to(device)
        else:
            # If no features are available, initialize the centroid to a zero vector
            centroids[class_idx] = torch.zeros(model.output_dim).to(device)
    return centroids
def ncm_loss(features, labels, centroids):
    loss = 0.0
    for feature, label in zip(features, labels):
        centroid = centroids[label.item()]
        loss += torch.norm(feature - centroid, p=2)
    return loss / len(features)

In [50]:
from sklearn.metrics import confusion_matrix
class_features = {class_idx: [] for class_idx in range(len(classes_dir))}
def train_modified(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    # centroids = {class_idx: torch.zeros(model.output_dim).to(device) for class_idx in range(len(classes_dir))}
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the optimizer gradients
            optimizer.zero_grad()

            # Forward pass
            features = model(inputs) 
            print(features.shape)
            for feature, label in zip(features, labels):
                class_features[label.item()].append(feature.cpu().detach())
            _, preds = torch.max(features, 1)
            centroids = calculate_centroids(class_features)
            loss = ncm_loss(features, labels, centroids)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy
        train_loss = running_loss / len(train_dataset)
        train_acc = running_corrects.double() / len(train_dataset)

       
all_labels, all_preds=train_modified(newmodel, train_loader, val_loader, criterion, optimizer, num_epochs=5)

torch.Size([32, 512, 1, 1])


AttributeError: 'ResNet' object has no attribute 'output_dim'

In [ ]:
class_centroids = {}
for class_idx, features in class_features.items():
    class_centroids[class_idx] = np.mean(features, axis=0)

# Convert centroids dictionary to a single array for easier use
centroids_array = np.array([class_centroids[class_idx] for class_idx in sorted(class_centroids.keys())])

print(f"Centroids shape: {centroids_array.shape}")  # Should print (num_classes, 512)
